<a href="https://colab.research.google.com/github/KamranUmer/Cohere-and-Pinecone/blob/main/Cohere_AI's_LLM_for_Semantic_Search_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Search with Cohere and Pinecone

In this notebook we will demonstrate how to perform semantic search for identifying similar or duplicate questions using Cohere and Pinecone.

![Steps in semantic search process](https://raw.githubusercontent.com/pinecone-io/examples/master/integrations/cohere/assets/index_query_pinecone_cohere.png)

## Setup

We first need to setup our environment and retrieve API keys for Cohere and Pinecone. Let's start with our environment, we need HuggingFace *Datasets* for our data, and the Cohere and Pinecone clients:

In [ ]:
!pip install cohere pinecone-client datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 756.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


And sign up for an API key over at [Cohere](https://os.cohere.ai/) and [Pinecone](https://app.pinecone.io), you can enter the keys directly in the cell below.

In [ ]:
import os
!pip install python-dotenv


In [ ]:
from dotenv import load_dotenv, find_dotenv
_ =  load_dotenv(dotenv_path="/content/drive/MyDrive/Cohere AI's LLM for Semantic Search in Python/.env")
# apit =
COHERE_KEY  = os.environ['COHERE_KEY']
PINECONE_KEY  = os.environ['PINECONE_KEY']
environment = os.environ['environment']



## Create Embeddings

We can create sentence embeddings easily using Cohere. First, we import the Cohere client and initialize our connection using the API key we retrieved earlier.

In [ ]:
import cohere

co = cohere.Client(COHERE_KEY)

We will load the **T**ext **RE**trieval **C**onference (TREC) question classification dataset which contains 5.5K labeled questions. We will take the first 1K samples for this demo, but this can be scaled to millions or even billions of samples.

In [ ]:
from datasets import load_dataset

# load the first 1K rows of the TREC dataset
trec = load_dataset('trec', split='train[:1000]')
trec

Dataset({
    features: ['text', 'coarse_label', 'fine_label'],
    num_rows: 1000
})

In [ ]:
trec[0]

{'text': 'How did serfdom develop in and then leave Russia ?',
 'coarse_label': 2,
 'fine_label': 26}

We can then pass these questions to Cohere to create embeddings.

In [ ]:
embeds = co.embed(
    texts=trec['text'],
    model='small',
    truncate='LEFT'
).embeddings

We can check the dimensionality of the returned vectors, for this we will convert it from a list of lists to a Numpy array. We will need to save the embedding dimensionality from this to be used when initializing our Pinecone index later.

In [ ]:
import numpy as np

shape = np.array(embeds).shape
shape

(1000, 1024)

Here we can see the `1024` embedding dimensionality produced by Cohere's small model, and the `1000` samples we built embeddings for.

## Storing the Embeddings

Now that we have our embeddings we can move on to indexing them in the Pinecone vector database. Again, this is very simple, we just initialize our connection to Pinecone and then create a new index for storing the embeddings, making sure to specify that we would like to use the cosine similarity metric to align with Cohere's embeddings.

In [ ]:
import pinecone

pinecone.init(
    PINECONE_KEY,
    environment=environment  # find next to API key in console
)

index_name = 'cohere-pinecone-trec'

# if the index does not exist, we create it
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=shape[1],
        metric='cosine'
    )

# connect to index
index = pinecone.Index(index_name)

Now we can begin populating the index with our embeddings. Pinecone expects us to provide a list of tuples in the format *(id, vector, metadata)*, where the *metadata* field is an optional extra field where we can store anything we want in a dictionary format. For this example, we will store the original text of the embeddings.

While uploading our data, we will batch everything to avoid pushing too much data in one go.

In [ ]:
batch_size = 128

ids = [str(i) for i in range(shape[0])]
# create list of metadata dictionaries
meta = [{'text': text} for text in trec['text']]

# create list of (id, vector, metadata) tuples to be upserted
to_upsert = list(zip(ids, embeds, meta))

for i in range(0, shape[0], batch_size):
    i_end = min(i+batch_size, shape[0])
    index.upsert(vectors=to_upsert[i:i_end])

# let's view the index statistics
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.01,
 'namespaces': {'': {'vector_count': 1000}},
 'total_vector_count': 1000}

Perfect, we can see from `index.describe_index_stats` that we have a *1024-dimensionality* index populated with *1000* embeddings. The `indexFullness` metric tells us how full our index is, at the moment it is empty. Using the default value of one *p1* pod we can fit ~750K embeddings before the `indexFullness` reaches capacity. The [Usage Estimator](www.pinecone.io/pricing) can be used to identify the number of pods required for a given number of *n*-dimensional embeddings.

## Semantic Search

Now that we have our indexed vectors we can perform a few search queries. When searching we will first embed our query using Cohere, and then search using the returned vector in Pinecone.

In [ ]:
# query = "What caused the 1929 Great Depression?"

query = "what event occured at 1978"

# create the query embedding
xq = co.embed(
    texts=[query],
    model='small',
    truncate='LEFT'
).embeddings

print(np.array(xq).shape)

# query, returning the top 10 most similar results
res = index.query(xq, top_k=10, include_metadata=True)
res

(1, 1024)


{'matches': [{'id': '887',
              'metadata': {'text': 'What events happened January 26 , 1978 ?'},
              'score': 0.813806176,
              'values': []},
             {'id': '367',
              'metadata': {'text': 'What Jewish holiday saw the start of the '
                                   '1973 Mideast War ?'},
              'score': 0.524388194,
              'values': []},
             {'id': '969',
              'metadata': {'text': 'What are some of the significant '
                                   'historical events of the 1990s ?'},
              'score': 0.513988614,
              'values': []},
             {'id': '775',
              'metadata': {'text': 'What historical event happened in Dogtown '
                                   'in 1899 ?'},
              'score': 0.490166336,
              'values': []},
             {'id': '919',
              'metadata': {'text': 'What astronomical phenomenon takes place '
                                   'i

The response from Pinecone includes our original text in the `metadata` field, let's print out the `top_k` most similar questions and their respective similarity scores.

In [ ]:
for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['text']}")

0.81: What events happened January 26 , 1978 ?
0.52: What Jewish holiday saw the start of the 1973 Mideast War ?
0.51: What are some of the significant historical events of the 1990s ?
0.49: What historical event happened in Dogtown in 1899 ?
0.45: What astronomical phenomenon takes place in Jan. 1999 ?
0.45: What year did Germany sign its nonaggression pact with the Soviet Union ?
0.44: What year did the Vietnam War end ?
0.42: What year did Hitler die ?
0.40: When was the San Francisco fire ?
0.39: What war did the Wanna-Go-Home Riots occur after ?


Looks good, let's make it harder and replace *"depression"* with the incorrect term *"recession"*.

In [ ]:
query = "What was the cause of the major recession in the early 20th century?"

# create the query embedding
xq = co.embed(
    texts=[query],
    model='small',
    truncate='LEFT'
).embeddings

# query, returning the top 10 most similar results
res = index.query(xq, top_k=10, include_metadata=True)

for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['text']}")

0.66: Why did the world enter a global depression in 1929 ?
0.61: When was `` the Great Depression '' ?
0.43: What crop failure caused the Irish Famine ?
0.43: What are some of the significant historical events of the 1990s ?
0.37: What were popular songs and types of songs in the 1920s ?
0.36: When did the Dow first reach ?
0.34: What war did the Wanna-Go-Home Riots occur after ?
0.34: What historical event happened in Dogtown in 1899 ?
0.33: What is considered the costliest disaster the insurance industry has ever faced ?
0.31: What was the education system in the 1960 's ?


And again.

In [ ]:
query = "Why was there a long-term economic downturn in the early 20th century?"

# create the query embedding
xq = co.embed(
    texts=[query],
    model='small',
    truncate='LEFT'
).embeddings

# query, returning the top 10 most similar results
res = index.query(xq, top_k=10, include_metadata=True)

for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['text']}")

0.71: Why did the world enter a global depression in 1929 ?
0.62: When was `` the Great Depression '' ?
0.40: What crop failure caused the Irish Famine ?
0.38: What are some of the significant historical events of the 1990s ?
0.38: When did the Dow first reach ?
0.35: What were popular songs and types of songs in the 1920s ?
0.33: What was the education system in the 1960 's ?
0.32: Give a reason for American Indians oftentimes dropping out of school .
0.31: What war did the Wanna-Go-Home Riots occur after ?
0.30: What historical event happened in Dogtown in 1899 ?


Looks great, our semantic search pipeline is clearly able to identify the meaning between each of our queries and return the most semantically similar questions from the already indexed questions.

Once we're done with the index we delete it to save resources:

---